# Машинное обучение, DS-поток, осень 2024
## Задание ML.12


**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.

**Баллы за задание**  
Задача 1 &mdash; **100 баллов**  
Задача 2 &mdash; **80 баллов**  

Всего &mdash; **180 баллов**  

In [ ]:
# Утилиты стандартной библиотеки
from functools import lru_cache
import os

# Библиотеки для анализа данных
import numpy as np

# Библиотеки для обработки изображений
from PIL import Image
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Библиотеки для обучения
import pytorch_lightning as pl
import albumentations as A

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchvision import models
from sklearn.model_selection import train_test_split

In [ ]:
# Bot check

# HW_ID: ds_ml12
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

### Данные

Мы предлагаем вам поработать с подмножеством датасета [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge) с Kaggle. Carvana &mdash; это онлайн магазин подержаннных aвтомобилей. Компания организовала соревнование для того, чтобы получить модель, с помощью которой получилось бы автоматически сегментировать авто на фотографии. Затем, используя маску для автомобиля, его можно было использовать для получения более интересных фотографий с автомобилем.


Мы не будем использовать использовать датасет целиком, так как он очень большой, а для наших целей хватит небольшой его части.

Скачайте архив с даными [здесь](https://drive.google.com/file/d/1UbXOTP6GIp1pp-LwwpfX1p8m6iaWXzM1/view?usp=drive_link). Как работать со внешними данными на Google Colab или Kaggle описано в предыдущем домашнем задании.

In [ ]:
zip_path = ...
data_path = ...
zip_path_bash = str(zip_path).replace(' ', '\ ')

In [ ]:
!unzip $zip_path_bash -d $data_path

Данные уже разделены на трейн и тест. В папках `train` и `train_masks` лежат изображения автомобилей и их маски соответственно. Аналогично для папок `test` и `test_masks`.

In [ ]:
!ls $data_path

Изображения и маски имеют одинаковое начало названия. Название файла маски отличается тем, что есть окончание `_mask`, а также сам файл имеет другой формат &mdash; gif вместо jpg. Так можно получить изображение и его маску из трейна.


In [ ]:
img_name = "0cdf5b5d0ce1_01.jpg"
splitted = img_name.split('.')
mask_name = splitted[0] + "_mask" + ".gif"

img_path = os.path.join(DATA_PATH, "train", img_name)
mask_path = os.path.join(DATA_PATH, "train_masks", mask_name)

Визуализируем изображение и маску.

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.title("Изображение")
plt.imshow(Image.open(img_path))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Маска")
plt.imshow(Image.open(mask_path))
plt.axis("off");

Исходные изображения довольно большие. Предлагаем вам работать с изображениями меньшего размера, а именно (160, 240) для уменьшения вычислительных затрат. Не забудьте сделать соответствующее преобразование.

In [ ]:
Image.open(img_path).size

### Задача 1. Простая сегментация
Вы построите свою собственную модель сегментации и проведете анализ ее работы.

#### 1. Обучение сегментации

Постройте модель типа U-Net. Вы можете использовать непосредственно первоначальную версию U-Net (рассматривали на лекции), либо ее модификацию, либо любую другую модель типа кодировщик-декодировщик с skip-connection конструкцией.

Обучите модель на тренировочной части данных. В качестве метрики качества используйте IoU. Визуализируйте изменение ошибки (loss) и метрики (IoU) на тренировочной и тестовой части данных.

Как быстро обучается модель? Как вы думаете, с чем это связано?

**Ответ:** ...

#### 2. Анализ модели
Выберите 10 изображений из тестовой части данных и посчитайте качество модели на них. Теперь поменяйте фон для изображений. Можно, например, поменять его цвет. Проверьте качество модели на новых изображениях.

Ответьте на вопрос, модель научилась сегментировать машины или фон?

**Ответ:** ...

**Вывод:**

### Задача 2. Сравнение лоссов сегментации
Будем работать с датасетом [CityScapes](https://www.cityscapes-dataset.com/). Этот датасет состоит из кадров съемки автомобиля, движущегося по улицам  Германии. Проверим, насколько хорошо с ним справляются предобученные модели со структурой, отличной от U-Net. Скачайте [архив](https://disk.yandex.ru/d/J8gPv_bEiK-_QQ) с данными с диска. В папках `images` и `masks` лежат изображения и маски автомобилей соответственно.



Определите датасет для CityScapes и даталоадеры.

Определите класс модуля lightning для обучения модели сегментации машин. В качестве модели используйте предобученную `fcn_resnet50` или `deeplabv3_resnet50` из `torchvision.models.segmentation` на ваш выбор. Особенностью их использования является то, что модели дают на выходе маски для 21 класса (как в COCO) в поле `out` (см. примеры в семинаре). В остальном модели не отличаются от тех, что мы использовали ранее.

Наша цель - сравнить работу 3 лоссов - Binary Cross Entropy, Dice Loss, Tversky Loss. Реализацию двух последних можно взять из библиотеки [monai](https://docs.monai.io/en/stable/losses.html).

В качестве метрики используйте IoU.

#### Визуализация до обучения

До обучения визуализируйте несколько предсказаний модели на тестовой части CityScapes. Покажите исходную картинку, предсказание модели и истинную маску. Не забудьте брать предсказания только по каналу машин.

#### Binary Cross Entropy

Обучите предобученную модель, используя BCE Loss. Сравните несколько предсказаний с истинными масками на тестовой выборке. Также после обучения (до завершения wandb run) выведите IoU на тесте с помощью `trainer.validate`.

#### Dice Loss

Сделайте то же самое для Dice Loss.

Судя по визуализации, чего больше у модели - False Negative или False Positive предсказаний?

#### Tversky Loss

Tversky Index является обобщением Dice Index для произвольных весов при FP и FN:

$$T_{\alpha, \beta} (y, \hat{y}) = \frac{\text{TP}}{\text{TP} + \alpha \ \text{FP} + \beta \ \text{FN}}$$

При этом $T_{\frac{1}{2}, \frac{1}{2}}$ соответсвует Dice Index; а $T_{1, 1} - $ Jaccard index (IoU).

Исходя из выводов про FN и FP подберите параметры $\alpha, \beta$ у Tversky Loss для модели. Обучите ее с таким лоссом. Улучшилась ли метрика? Как изменились предсказания визуально?

#### Визуализация логов обучения

Визуализируйте лосс и метрику на обучении и валидации на предыдущих запусках. Сравните и сделайте выводы, а также выводы по всей задаче.

**Выводы:**

